In [1]:
import torch
from x_transformers import XTransformer as PTXTransformer
from tf_x_transformers import XTransformer as TFXTransformer
from tf_fast_api import *

In [2]:
enc_depth = 6
dec_depth = 6
tie_token_emb = False

In [3]:
torch.manual_seed(1)
model = PTXTransformer(
    dim = 512,
    enc_num_tokens = 256,
    enc_depth = enc_depth,
    enc_heads = 8,
    enc_max_seq_len = 1024,
    dec_num_tokens = 256,
    dec_depth = dec_depth,
    dec_heads = 8,
    dec_max_seq_len = 1024,
    tie_token_emb = tie_token_emb      # tie embeddings of encoder and decoder
)
model.eval()
src = torch.randint(0, 256, (1, 1024))
src_mask = torch.ones_like(src).bool()
tgt = torch.randint(0, 256, (1, 1024))
tgt_mask = torch.ones_like(tgt).bool()


In [4]:
class TMP(keras.Model):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.model = TFXTransformer(
            dim = 512,
            enc_num_tokens = 256,
            enc_depth = enc_depth,
            enc_heads = 8,
            enc_max_seq_len = 1024,
            dec_num_tokens = 256,
            dec_depth = dec_depth,
            dec_heads = 8,
            dec_max_seq_len = 1024,
            tie_token_emb = tie_token_emb      # tie embeddings of encoder and decoder
        )
    def call(self,a,b,c,d):
        return self.model(a, b, src_mask = c, tgt_mask = d)
tf.keras.backend.set_learning_phase(0)
tfmodel = TMP()
tf_src = tf.constant(src.numpy())
tf_src_mask = tf.constant(src_mask.numpy())
tf_tgt = tf.constant(tgt.numpy())
tf_tgt_mask = tf.constant(tgt_mask.numpy())
tf_loss = tfmodel(tf_src, tf_tgt,tf_src_mask,tf_tgt_mask) # (1, 1024, 512)
tf_loss.item()

C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


6.070344924926758

In [5]:
tf_path = "./ckpts/checkpoint"
tfmodel.save_weights(tf_path)  
init_vars = tf.train.list_variables(tf_path)
names = []
arrays = []
layer_depth = []
for full_name, shape in init_vars:
    if full_name == "_CHECKPOINTABLE_OBJECT_GRAPH" or full_name[0] in ["global_step", "save_counter"]:
        print(f"Skipping non-model layer {full_name}")
        continue
    names.append(full_name.replace(".ATTRIBUTES/VARIABLE_VALUE","").replace("model/",""))
    array = tf.train.load_variable(tf_path, full_name)
    array = torch.from_numpy(array)
    if "kernel" in names[-1]:
        array = array.T
    arrays.append(array)
mmap = dict(zip(names,arrays))

Skipping non-model layer _CHECKPOINTABLE_OBJECT_GRAPH


In [6]:
pt_model = model.encoder.attn_layers
prefix = "encoder/attn_layers"
for i,layer in enumerate(pt_model.layer_types): 
    is_last = i==len(pt_model.layer_types)-1
    i = str(i)
    if layer == "f" :
        if not is_last:
            getattr(getattr(pt_model.layers,i),"0").weight.data = mmap[f"{prefix}/tf_layers/{i}/0/gamma/"]
            getattr(getattr(pt_model.layers,i),"0").bias.data = mmap[f"{prefix}/tf_layers/{i}/0/beta/"]

        getattr(getattr(getattr(getattr(pt_model.layers,i),"1").net,"0"),"0").weight.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-0/kernel/"]
        getattr(getattr(getattr(getattr(pt_model.layers,i),"1").net,"0"),"0").bias.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-0/bias/"]
        getattr(getattr(getattr(pt_model.layers,i),"1").net,"2").weight.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-1/kernel/"]
        getattr(getattr(getattr(pt_model.layers,i),"1").net,"2").bias.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-1/bias/"]

    elif layer =="a" or layer=="c":
        getattr(getattr(pt_model.layers,i),"0").weight.data = mmap[f"{prefix}/tf_layers/{i}/0/gamma/"]
        getattr(getattr(pt_model.layers,i),"0").bias.data = mmap[f"{prefix}/tf_layers/{i}/0/beta/"]

        getattr(getattr(pt_model.layers,i),"1").to_q.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_q/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_k.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_k/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_v.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_v/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_out.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_out/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_out.bias.data = mmap[f"{prefix}/tf_layers/{i}/1/to_out/bias/"]




In [7]:
pt_model = model.decoder.net.attn_layers
prefix = "decoder/net/attn_layers"
for i,layer in enumerate(pt_model.layer_types): 
    is_last = i==len(pt_model.layer_types)-1
    i = str(i)
    if layer == "f" :
        if not is_last:
            getattr(getattr(pt_model.layers,i),"0").weight.data = mmap[f"{prefix}/tf_layers/{i}/0/gamma/"]
            getattr(getattr(pt_model.layers,i),"0").bias.data = mmap[f"{prefix}/tf_layers/{i}/0/beta/"]

        getattr(getattr(getattr(getattr(pt_model.layers,i),"1").net,"0"),"0").weight.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-0/kernel/"]
        getattr(getattr(getattr(getattr(pt_model.layers,i),"1").net,"0"),"0").bias.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-0/bias/"]
        getattr(getattr(getattr(pt_model.layers,i),"1").net,"2").weight.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-1/kernel/"]
        getattr(getattr(getattr(pt_model.layers,i),"1").net,"2").bias.data = mmap[f"{prefix}/tf_layers/{i}/1/net/layer_with_weights-1/bias/"]

    elif layer =="a" or layer=="c":
        getattr(getattr(pt_model.layers,i),"0").weight.data = mmap[f"{prefix}/tf_layers/{i}/0/gamma/"]
        getattr(getattr(pt_model.layers,i),"0").bias.data = mmap[f"{prefix}/tf_layers/{i}/0/beta/"]

        getattr(getattr(pt_model.layers,i),"1").to_q.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_q/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_k.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_k/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_v.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_v/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_out.weight.data = mmap[f"{prefix}/tf_layers/{i}/1/to_out/kernel/"]
        getattr(getattr(pt_model.layers,i),"1").to_out.bias.data = mmap[f"{prefix}/tf_layers/{i}/1/to_out/bias/"]




In [8]:
# names

In [9]:
model.encoder.token_emb.weight.data  = mmap['encoder/token_emb/embeddings/']
model.encoder.pos_emb.emb.weight.data = mmap['encoder/pos_emb/emb/embeddings/']
try:
    model.decoder.net.token_emb.weight.data = mmap['decoder/net/token_emb/embeddings/']
    model.decoder.net.pos_emb.emb.weight.data= mmap['decoder/net/pos_emb/emb/embeddings/']
except:
    pass
model.encoder.norm.weight.data = mmap['encoder/norm/gamma/']
model.encoder.norm.bias.data = mmap['encoder/norm/beta/']

model.decoder.net.norm.weight.data  = mmap['decoder/net/norm/gamma/']
model.decoder.net.norm.bias.data  = mmap['decoder/net/norm/beta/']
model.decoder.net.to_logits.weight.data  = mmap['decoder/net/to_logits/kernel/']
model.decoder.net.to_logits.bias.data  = mmap['decoder/net/to_logits/bias/']


In [10]:
with torch.no_grad():
    loss1 = model(src, tgt, src_mask = src_mask, tgt_mask = tgt_mask) # (1, 1024, 512)

In [11]:
(loss1 - torch.tensor(tf_loss.numpy())).abs().mean()

tensor(4.7684e-07)

In [12]:
(loss1 - torch.tensor(tf_loss.numpy())).abs().max()

tensor(4.7684e-07)